### Compare cerebellar volumes across datasets

#### Datasets
- NIMHANS
- QPN


In [ ]:
import sys
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nibabel as nib
from nilearn import datasets, surface, plotting
from pathlib import Path

In [ ]:
dataset = "qpn"
current_release = "Jan_2024"
maget_verion = "1.0.0"
session = "ses-01"

dataset_dir = f"/home/nikhil/projects/Parkinsons/{dataset}/"
tabular_dir = f"{dataset_dir}/releases/{current_release}/tabular/"

# Current nipoppy manifest
manifest_csv = f"{tabular_dir}/manifest.csv"

# demographics
demographics_csv = f"{tabular_dir}/demographics/demographics.csv"

# imaging derivatives
maget_dir = f"{dataset_dir}/derivatives/maget_brain/v{maget_verion}/"
CB_vol_csv = f"{maget_dir}/output/ses-01/cerebellum_segmentations/cerebellar_volumes.csv"

# save dirs
results_dir = f"/home/nikhil/projects/Parkinsons/neuro_arch/analysis/IDP/{dataset}/{current_release}/agg_dfs/"
figs_dir = f"/home/nikhil/projects/Parkinsons/neuro_arch/analysis/IDP/{dataset}/{current_release}/figs/"

# Create dirs for results if they don't exist
Path(f"{results_dir}").mkdir(parents=True, exist_ok=True)
Path(f"{figs_dir}").mkdir(parents=True, exist_ok=True)

In [ ]:
from enum import Enum
class my_colors(Enum):
    CONTROL = "#8d99ae"
    PD = "#e63946"
    
color_list = [my_colors.PD.value, my_colors.CONTROL.value,]
palette = sns.color_palette(palette=color_list) #sns.husl_palette()

sns.palplot(palette)

In [ ]:
def quick_QC(df, check_cols, min_val, max_val, index_col="participant_id"):
    """Checks for NaNs and out of range outliers """
    
    if index_col in df.columns:
        # check NaNs
        nan_participants = df[df[check_cols].isna().any(axis=1)][index_col].values
        n_nans = len(nan_participants)

        # check range
        outlier_participants = df[df[check_cols].apply(lambda x: (x < min_val) | (x > max_val)).any(axis=1)][index_col].values
        n_outliers = len(outlier_participants)

        print(f"found {n_nans} NaNs and {n_outliers} outliers")
        return list(nan_participants), list(outlier_participants)
    
    else:
        print(f"Provide an index column")
        return None, None

def merge_CB_demo_df(CB_df, demo_df, cerebellar_lobules):
    left_lobules = list("L_" + pd.Series(cerebellar_lobules))
    right_lobules = list("R_" + pd.Series(cerebellar_lobules))
    lh_lobule_dict = dict(zip(left_lobules,cerebellar_lobules))
    rh_lobule_dict = dict(zip(right_lobules,cerebellar_lobules))

    CB_df["bids_id"] = CB_df["Subject"].str.rsplit("/", n=1, expand=True)[1].str.split("_", expand=True)[0]
    CB_df["participant_id"] = CB_df["bids_id"].str.split("-", expand=True)[1]

    # merge with demo
    CB_demo_df = pd.merge(CB_df, demo_df, on="participant_id", how="inner")
    CB_demo_df = CB_demo_df.drop(columns=["Subject"])

    demo_cols = list(set(CB_demo_df.columns) - set(left_lobules) - set(right_lobules))

    left_CB_df = CB_demo_df[demo_cols + left_lobules].copy()
    left_CB_df["hemi"] = "left"
    right_CB_df = CB_demo_df[demo_cols + right_lobules].copy()
    right_CB_df["hemi"] = "right"

    # add total CB vol column
    left_CB_df["total_CB_vol"] = left_CB_df[left_lobules].sum(axis=1)
    right_CB_df["total_CB_vol"] = right_CB_df[right_lobules].sum(axis=1)

    # rename columns to stack vertically
    left_CB_df = left_CB_df.rename(columns=lh_lobule_dict)
    right_CB_df = right_CB_df.rename(columns=rh_lobule_dict)
    
    CB_demo_df = pd.concat([left_CB_df, right_CB_df], axis=0)

    return CB_demo_df

### Demographic data

In [ ]:
group_col = "group_at_screening"

# useful groups:
useful_groups = ["PD", "control"]

demographics_df = pd.read_csv(demographics_csv)
demographics_df = demographics_df[demographics_df["visit"] == "MRI_v1"]

participants = demographics_df["participant_id"].unique()
n_participants = len(participants)
print(f"n_participants: {n_participants}")

# replace column values
demographics_df[group_col] = demographics_df[group_col].replace({"PD   (Parkinson's Disease)/Maladie de Parkinson":"PD", 
                                                               "Healthy control/Contrôle": "control",
                                                               "PPS (Parkinson Plus Syndrome)/PPS (Syndrome Parkinson Plus)": "PPS"})

group_counts = demographics_df.groupby([group_col])["participant_id"].count()
print(f"group counts: {group_counts}")

# filter out useful groups
demographics_df = demographics_df[demographics_df[group_col].isin(useful_groups)]
demographics_df = demographics_df.rename(columns={group_col:"group"})

demographics_df.head()

### Get merged cerebellar volumes + demo df

In [ ]:
cerebellar_lobules = ['I_II', 'III', 'IV', 'V', 'VI', 'Crus_I', 'Crus_II','VIIB', 'VIIIA', 'VIIIB', 'IX', 'X', 'CM']
demo_cols = ["participant_id", "group","visit"]
demographics_df = demographics_df[demo_cols].copy()

CB_vols_df = pd.read_csv(f"{CB_vol_csv}")
n_CB_participants = CB_vols_df["Subject"].nunique()
print(f"n_CB_participants: {n_CB_participants}")

CB_demo_vols_df = merge_CB_demo_df(CB_vols_df, demographics_df, cerebellar_lobules)
CB_demo_vols_df.head()

### Remove outliers
 This is structure specific (need to be QCed visually)

In [ ]:
min_vol_thresh_list = [25, 400, 900, 2000, 4500, 6000, 4000, 2000, 2000, 1500, 1500, 200, 4000, 32000]
max_vol_thresh_list = [200, 1300, 3500, 6000, 11000, 18000, 13000, 7000, 8000, 5000, 6000, 800, 13000, 80000]
outlier_min_thesh_dict = dict(zip(cerebellar_lobules, min_vol_thresh_list))
outlier_max_thesh_dict = dict(zip(cerebellar_lobules, max_vol_thresh_list))

print("Removing outliers")
for roi, thresh in outlier_min_thesh_dict.items():
    n_participants = CB_demo_vols_df["participant_id"].nunique()
    print(f"roi: {roi}, n_participants: {n_participants}")
    CB_vol_df = CB_demo_vols_df[CB_demo_vols_df[roi] > thresh].copy()
    n_participants = CB_demo_vols_df["participant_id"].nunique()
    print(f"n_participants after outlier removal: {n_participants}")

for roi, thresh in outlier_max_thesh_dict.items():
    n_participants = CB_demo_vols_df["participant_id"].nunique()
    print(f"roi: {roi}, n_participants: {n_participants}")
    CB_vol_df = CB_demo_vols_df[CB_demo_vols_df[roi] < thresh].copy()
    n_participants = CB_demo_vols_df["participant_id"].nunique()
    print(f"n_participants after outlier removal: {n_participants}")


### Plots

In [ ]:
save_fig = False
# CB_demo_vols_df = CB_demo_vols_df.drop(columns=["bids_id"])
CB_vol_df_melt = CB_demo_vols_df.melt(
    id_vars=demo_cols + ["bids_id", "hemi"],
    var_name="ROI", 
    value_name="volume",
)

plot_df = CB_vol_df_melt.copy()

sns.set_theme(font_scale=4)
with sns.axes_style("whitegrid"):
    g = sns.catplot(y="volume",x="hemi", hue="group", col="ROI",kind="box", col_wrap=7, #col_order=hemi_roi_list,
    palette=palette, data=plot_df, aspect=1, height=10, sharey=False)
    # g.tick_params(axis='x', rotation=90, labelsize=14)
    
if save_fig:
    g.savefig(f"{figs_dir}/CB_vol.png")

In [ ]:
import numpy as np

